In [1]:
images = {
            'company_logo': 'static/images/company_logo.png',
            'phone_icon': 'static/images/phone_icon.png',
            'email_icon': 'static/images/email_icon.png',
            'web_icon': 'static/images/web_icon.png'
        }

print(images.items())

dict_items([('company_logo', 'static/images/company_logo.png'), ('phone_icon', 'static/images/phone_icon.png'), ('email_icon', 'static/images/email_icon.png'), ('web_icon', 'static/images/web_icon.png')])


In [7]:
print(list(range(4)))
lt = list(range(4))
print(lt[0:2])
min(lt)
lt[:-1]


[0, 1, 2, 3]
[0, 1]


[0, 1, 2]